This notebook contains code of lexical ans syntactic probing of BERT.

#### Experiment: Distribution of jokes and non-jokes with the same vocabulary in the BERT-family models' vector space.

First of all, jokes and non-jokes were distributed in the vector space of the BERT family models ALBERT, DistilBERT, multilingual BERT, and Roberta-base. The initial dataset comprised 15,000 jokes and 15,000 non-jokes with similar vocabulary. They were vectorised using each of the above models, after which the dimension was reduced to 2 using the UMAP algorithm. Non-humorous texts are marked in green, humorous ones in red.


##### Loading and vectorizing data

In [ ]:

import string
import re
import numpy as np
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import os
import pandas as pd
rom transformers import BertModel, BertPreTrainedModel, BertTokenizer
import matplotlib.pyplot as plt
import time
import datetime
import random
import re
import matplotlib
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

!ls /content/gdrive/MyDrive/trained_models


In [ ]:
ffrom transformers import BertModel, BertPreTrainedModel, BertTokenizer, AutoTokenizer, AutoModel, DistilBertModel, RobertaTokenizer, RobertaModel, TFBertModel, TFBertTokenizer, AlbertModel

#tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

#bert_model = DistilBertModel.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype=torch.float32)
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#bert_model = RobertaModel.from_pretrained('roberta-base')
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
#bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )

In [ ]:
dt_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/jokes_exp_triggers_24k.json", lines= True)
dt_reddit_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/reddit_jokes.json")

In [ ]:
jokes = [str(i+ " " + u).lower() for i,u in zip(dt_reddit_jokes["title"][:20000], dt_reddit_jokes["body"][:20000]) if len(str(i+ " " + u)) <= 256]

In [ ]:
jokes = jokes[:15000]

In [ ]:
j_vocab = [u.lower() for i in jokes for u in i.replace(".", " ").replace("’s", " is ")
                                              .replace("!", " ").replace("'s", " is ")
                                              .replace(",", " ").replace("–", "")
                                              .replace(":", "").replace("*", "")
                                              .replace("\n"," ").replace("-","")
                                              .replace("?", " ").split(" ")]

len(j_vocab)

In [ ]:
dt_n_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/length_comp_news.json", lines=True)
dt_n_sec = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/unfiltered_news_v3.json", lines=True)
dt_n_thi = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Tweets.csv")
dt_n_fou = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Twitter_Data.csv")
dt_f_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/eng_literature.json", lines=True)
dt_n_five = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/News_Category_Dataset_v3.json", lines=True)
dt_n_six = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/bbc_news.csv")
dt_n_sev = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/abcnews-date-text.csv")
dt_n_eig = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/uci-news-aggregator.csv")
dt_n_nine = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/exp_trig_data_news_filtered.json")

In [ ]:
news = [i for i in dt_n_one["headline"] if len(i) <= 256] +  [i for i in dt_n_one["news_preproc"] if len(i) <= 256] +  [i for i in dt_n_sec["headline"] if len(i) <= 256] + [i for i in dt_n_sec["short_description"] if len(i) <= 256] +  [str(i) for i in dt_n_fou["clean_text"].fillna("no") if i != None and len(i) <= 256] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if len(i) <= 256 and i != None] + [i for i in dt_n_thi["text"].fillna("no") if len(i) <= 256] + [i for i in dt_n_five["headline"] if len(i) <= 256] + [i for i in dt_n_five["short_description"] if len(i) <= 256]+ [i for i in dt_n_six["description"] if len(i) <= 256] + [i for i in dt_n_six["title"] if len(i) <= 256] + [i for i in dt_n_sev["headline_text"] if len(i) <= 256] + [i for i in dt_n_eig["TITLE"] if len(i) <= 256]  +  [i for i in dt_n_nine["headline"] if  len(i) <= 256]
len(news)

In [ ]:
#resultss = []
for item in tqdm(news, total=len(news)):
  new = item.lower().replace(",", " ").replace(".", " ").replace('"', " ").replace('“', "").replace("!", " ").replace("?", " ").replace(":", " ").replace("\n", " ").split(" ")
  comparison = list(set(j_vocab) & set(new))
  if len(new) - len(comparison) == 0:
    resultss.append(item)

In [ ]:
results = results[:15000]

In [ ]:
news = [i for i in results ]
n_vocab = list(set([u.lower() for i in news for u in i.replace(",", " ").replace(".", " ").replace('"', " ").replace('“', "").replace("!", " ").replace("?", " ").replace(":", " ").replace("\n", " ").split(" ")]))


In [ ]:
def bert_embed(sent):
  text = sent.lower().replace('"', '') # убираем лишние кавычки и приводим к нижнему регистру
  marked_text = "[CLS] " + text + " [SEP]" # добавляем начальные метки, указываем границы анализируемого предложения

  # делим предложения на токены
  tokenized_text = tokenizer.tokenize(marked_text)
  encodings = tokenizer(marked_text) # получаем начальные  представления

  # Сопоставляем токены с их словарными индексами.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Посмотреть на токены и индексы.
  #for tup in zip(tokenized_text, indexed_tokens):
  #    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)

      # Оценка модели вернет различное количество объектов в зависимости от того, как она была настроена в предыдущем вызове `from_pre trained`.
      # В этом случае, поскольку мы установили ' output_hidden_states = True, третьим элементом будут скрытые состояния из всех слоев.
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2] # берем скрытые состояния

  token_embeddings = torch.stack(hidden_states, dim=0) # предобрабатываем полученные скрытые состояния: объединяем последовательность тензоров вдоль новой новой размерности
  token_embeddings = torch.squeeze(token_embeddings, dim=1) # Возвращает новый тензор, где удалена часть входных данных с размером 1.
  token_embeddings = token_embeddings.permute(1,0,2) # перестановка данных для легкого изменения размеров тензора.

  token_embeddings.size() # векторные представления имеют размерность 22  x 768
  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers. # берем векторные представления с последних 4 слоев и суммируем их.
      sum_vec = torch.sum(token[-4:], dim=0)

      # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)

  vector = sum(token_vecs_sum) / len(token_vecs_sum)
  return vector

In [ ]:
from numpy import dot
from numpy.linalg import norm

#cos_sim = dot(a, b)/(norm(a)*norm(b))
missed = []
embedded = []
for joke in tqdm(jokes, total=len(jokes)):
  vector = bert_embed(sent = joke)
  embedded.append({"text": joke, "label":0, "vector": vector})

for new in tqdm(resultss, total=len(resultss)):
  vector = bert_embed(sent = new)
  embedded.append({"text": new, "label":1, "vector": vector})




In [ ]:
#### сохранение данных

import pickle
import torch

a = torch.rand(3,4,5)

# save
with open('/content/gdrive/MyDrive/trained_models/filename_jokes_news_same_30000_finn.pickle', 'wb') as handle:
    pickle.dump(embedded, handle)

# open
#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

##### Applying UMAP algorithm

In [ ]:
import pickle

with open('/content/gdrive/MyDrive/trained_models/filename_jokes_news_same_30000_finn.pickle', 'rb') as handle:
    words_embed = pickle.load(handle)



In [ ]:
jokes = [i for i in embedded if i["label"] == 0 ]
j_vocab = list(set([u for i in jokes for u in i['text'].replace(",", " ").replace('"', " ").replace('“', "").replace(".", " ").replace("!", " ").replace("?", " ").replace(":", " ").replace("\n", " ").split(" ")]))

In [ ]:
news = [i["text"] for i in embedded if i["label"] == 1 ]
n_vocab = list(set([u.lower() for i in news for u in i.replace(",", " ").replace(".", " ").replace('"', " ").replace('“', "").replace("!", " ").replace("?", " ").replace(":", " ").replace("\n", " ").split(" ")]))
print(len(n_vocab))

In [ ]:
!pip install umap-learn

In [ ]:
#UMAP-LEARN
import umap

umap_text = umap.UMAP(n_components=2, random_state=100)
umap_reduced = umap_text.fit_transform([i["vector"].numpy() for i in embedded])

In [ ]:

####Картинка UMAP
fig, ax = plt.subplots(1,1, figsize= (13, 15))
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(umap_reduced[:, 0], umap_reduced[:, 1], c=["red" if i["label"] == 0 else "green" for i in embedded], s=100, alpha=0.1) # train_b[:, 0], train_b[:, 1], c=list(k_means_bert) - нужно заменить, чтобы получить виз-цию результатов для Берта.


#plt.xlim(xmin=0, xmax=30)
# only show 100 - 1600 on y-axis.
#plt.ylim(ymin=0, ymax=30)

ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

plt.show()

#### Experiment: Assessing the impact of word polysemy on the perception of text as humorous

At the first stage of the experiment, the context-specific vectorisation of words was analysed: was a particular word present in the joke or not. To establish that, selected jokes and news with the same vocabulary comprising both polysemic and non-polysemic words were taken. 8 polysemic nouns (school, life, right, home) and verbs (have, hear, take, tell, say) were selected for the analysis, along with 8 non-polysemic nouns (animal, murder, mom, police, divorce) and verbs (power, rain, rid, own, thank). Words’ polysemy was determined by the number of synsets in their description in WordNet. The part of speech was determined using a spaCy library English-language model. Humorous and non-humorous texts were selected according to the word’s part of speech. The selected jokes and non-jokes were vectorised using the “bert-base-uncased” model and processed by the UMAP algorithm.  Humorous texts are marked in red, non-humorous ones in green.

In [ ]:

import string
import re
import numpy as np
from numpy import array, argmax, random, take
from tqdm import tqdm

import json
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import os
import pandas as pd
from transformers import BertModel, BertPreTrainedModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertPreTrainedModel
from nltk.corpus import stopwords
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

!ls /content/gdrive/MyDrive/trained_models

In [ ]:
dt_n_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/length_comp_news.json", lines=True)
dt_n_sec = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/unfiltered_news_v3.json", lines=True)
dt_n_thi = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Tweets.csv")
dt_n_fou = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Twitter_Data.csv")
dt_f_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/eng_literature.json", lines=True)
dt_n_five = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/News_Category_Dataset_v3.json", lines=True)
dt_n_six = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/bbc_news.csv")
dt_n_sev = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/abcnews-date-text.csv")
dt_n_eig = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/uci-news-aggregator.csv")
dt_n_nine = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/exp_trig_data_news_filtered.json")

In [ ]:
import spacy

model = spacy.load("en_core_web_sm")

ms_razbor = []
for i in  school_jokes:
  for token in model(i):
    if token.pos_ == "NOUN":
      ms_razbor.append({"text":i, "word": token.text, "lemma": token.lemma_, "pos": token.pos_})
    else:
      continue

In [ ]:
dt_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/jokes_exp_triggers_24k.json", lines= True)
dt_reddit_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/reddit_jokes.json")

In [ ]:
dt_reddit_jokes["joke"] = [str(i+ " " + u) for i,u in zip(dt_reddit_jokes["title"], dt_reddit_jokes["body"])]

In [ ]:
words = ["school", "home", "right", "life"] #time
word = words[0]

In [ ]:
school_jokes = [i for i in dt_jokes["joke"] if word in i and len(i) <= 256] + [i.lower() for i in dt_reddit_jokes["joke"] if word in i and len(i) <= 256]
school_jokes = school_jokes[:500]
j_vocab = [u.lower() for i in school_jokes for u in i.replace(".", " ").replace("’s", " is ")
                                              .replace("!", " ").replace("'s", " is ")
                                              .replace(",", " ").replace("–", "")
                                              .replace(":", "").replace("*", "")
                                              .replace("\n"," ").replace("-","")
                                              .replace("?", " ").split(" ")]

len(school_jokes)

In [ ]:
school_news = [i for i in dt_n_one["headline"] if word in i.lower() and len(i) <= 256] + [i for i in dt_n_one["news_preproc"] if word in i.lower() and len(i) <= 256] + [i for i in dt_n_sec["headline"] if word in i.lower() and len(i) <= 256] +  [i for i in dt_n_sec["short_description"] if word in i.lower() and len(i) <= 256] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None and len(i) <= 256] + [i for i in dt_n_sec["headline"] if word in i.lower() and len(i) <= 256] +  [i for i in dt_n_sec["short_description"] if word in i.lower() and len(i) <= 256] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and  len(i) <= 256] + [i for i in dt_n_sec["headline"] if word in i.lower() and len(i) <= 256] +  [i for i in dt_n_sec["short_description"] if word in i.lower() and len(i) <= 256] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_thi["text"].fillna("no") if word in i.lower()] + [i for i in dt_n_five["headline"] if word in i.lower()] + [i for i in dt_n_five["short_description"] if word in i.lower()]+ [i for i in dt_n_six["description"] if word in i.lower()] + [i for i in dt_n_six["title"] if word in i.lower()] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [i for i in dt_n_sec["headline"] if word in i.lower()] +  [i for i in dt_n_sec["short_description"] if word in i.lower()] + [str(i) for i in dt_n_fou["clean_text"].fillna("no") if word in i and i != None] + [i for i in dt_n_thi["text"].fillna("no") if word in i.lower()] + [i for i in dt_n_five["headline"] if word in i.lower()] + [i for i in dt_n_five["short_description"] if word in i.lower()]+ [i for i in dt_n_six["description"] if word in i.lower()] + [i for i in dt_n_sev["headline_text"] if word in i.lower()] + [i for i in dt_n_eig["TITLE"] if word in i.lower() and len(i) <= 256]  + [i for i in dt_n_nine["headline"] if word in i.lower() and len(i) <= 256]
len(school_news)

In [ ]:
results = []
for item in tqdm(school_news, total=len(school_news)):
  new = item.lower().split(" ")
  comparison = list(set(j_vocab) & set(new))
  if len(new) - len(comparison) == 0:
    results.append(item)

In [ ]:
all_data = [{"text":i, "text_preproc": i.lower(), "label": 1} for i in results[:len(school_jokes)]] + [ {"text":i, "text_preproc": i.lower(), "label": 0 } for i in school_jokes]
len(all_data)

In [ ]:
def bert_embed(sent):
  text = sent.lower().replace('"', '')
  marked_text = "[CLS] " + text + " [SEP]"

  tokenized_text = tokenizer.tokenize(marked_text)
  encodings = tokenizer(marked_text)

  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)


      hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  token_embeddings.size()
  token_vecs_sum = []


  for token in token_embeddings:


      sum_vec = torch.sum(token[-4:], dim=0)

      token_vecs_sum.append(sum_vec)

  vector = sum(token_vecs_sum) / len(token_vecs_sum)
  return vector

In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertTokenizer

bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )

In [ ]:
from numpy import dot
from numpy.linalg import norm

#cos_sim = dot(a, b)/(norm(a)*norm(b))
missed = []
embedded = []
for diction in tqdm(all_data, total=len(all_data)):
  if len(diction["text_preproc"]) > 512:
    missed.append(diction)
  else:
    vector = bert_embed(sent =diction["text_preproc"])
    diction["vector"]= vector#
    embedded.append(diction)




In [ ]:
len(missed)

In [ ]:
import heapq
from sklearn.cluster import AffinityPropagation, MeanShift
from scipy.spatial import distance

all_res = []
count_clusters = []

clustering = MeanShift(bandwidth = 30).fit([item["vector"].numpy() for item in embedded]) #######  проблема в снижении размерности???????
centroids = clustering.cluster_centers_


In [ ]:
all_res = []
for item, cluster in zip(embedded, clustering.labels_ ):
  item["cluster"] =cluster
  all_res.append(item)

In [ ]:
m_res = pd.DataFrame(all_res)

In [ ]:
!pip install umap-learn

In [ ]:
#UMAP-LEARN
import umap

umap_text = umap.UMAP(n_components=2, random_state=100)
umap_reduced = umap_text.fit_transform([item["vector"].numpy() for item in embedded])

In [ ]:
fig, ax = plt.subplots(1,1, figsize= (13, 15))
#ax = fig.add_subplot(111, projection='3d')
ax.scatter(umap_reduced[:,0], umap_reduced[:,1], c=[ "red" if i["label"]==0 else "green" for i in embedded], alpha=0.5)
plt.show()


#### Experiment with polysemic and non-polysemic word and its contexts
We conducted an experiment to assess the effectiveness of clustering contextualised BERT vectors of polysemic and non-polysemic words. With the help of WordNet, 15 polysemic nouns with more than one synset, and 15 non-polysemic nouns with a single synset have been selected from 100 joke punchlines. 500 fragments were selected from news headlines and texts, and from fiction books in English, containing the same polysemic words. The contexts for each of the selected words were vectorised using a “bert-base-uncased” model. The resulting vector representations were clustered for each word individually using the MeanShift algorithm. For each word was also calculated the cosine distance between the centroids of its context of the use clusters, to identify the clusters most removed from each other.


#####Loading and preparation of non-jokes (news and fiction literature), and NOUNS from 100 jokes

In [ ]:
!pip install transformers

In [ ]:
#Загрузка библиотек
import string
import re
import numpy as np
from numpy import array, argmax, random, take
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import json
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import torch
import os
import pandas as pd
from transformers import BertModel, BertPreTrainedModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch import nn
from torch.nn import BCEWithLogitsLoss
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from nltk.corpus import stopwords
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from nltk import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
stop_w = stopwords.words('english')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

!ls /content/gdrive/MyDrive/trained_models

In [ ]:
stop_symbols = [',', ".", ";", ":", "!", "'", '"', "/", '\\', '-', '–' '(', ")", '=', '+', "—", '’', '#', "?", "...", " "]
test_df = pd.read_csv("/content/gdrive/MyDrive/trained_models/exp_triggers_test_df_bigdata_38k.csv")
test_df["label_res"] =  test_df["label"].apply(lambda x: "joke" if x == 0 else "non-joke")
test_df["text"] =  [ i.replace('...', '.').replace("  ", ' ') for  i in test_df["text"]]
test_df["text_unpreproc"] = test_df["text_unpreproc"].fillna(test_df["text"])
test_df["text_unpreproc"] = test_df["text_unpreproc"].apply(lambda i: str(i + " .") if i[-1] not in stop_symbols else i)
test_df["text"] =  test_df["text"].apply(lambda i: None if i[-1] not in stop_symbols else i)
test_df["text"] = test_df["text"].fillna(test_df["text_unpreproc"])
test_df["text"]  = [i.lower().replace('.', ' .').replace('!', ' !').replace('?', ' ?') for i in test_df["text"]]
#test_df["text"].apply(lambda i: str(i + ".") if i[-1] not in stop_symbols else i)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=2)
test_df= test_df.groupby(["label"]).head(100)
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df = test_df[test_df["label"] == 0].reset_index(drop=True).reindex()

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
import spacy
from nltk.corpus import wordnet as wn

model = spacy.load("en_core_web_sm")

ms_razbor = []
for i in test_df["punchline"]:
  for token in model(i):
    if token.pos_ == "NOUN":
      count_m = len(wn.synsets(token.lemma_, pos=wn.NOUN))
      if count_m ==1:
        ms_razbor.append({"punchline":i, "word": token.text, "lemma": token.lemma_, "pos": token.pos_, "count_word_net": 1})
      elif count_m > 1:
        ms_razbor.append({"punchline":i, "word": token.text, "lemma": token.lemma_, "pos": token.pos_, "count_word_net": 2})
    else:
      continue

In [ ]:
def preproc(text):
  text = re.sub('<HTML>\n\n<HEAD>\n\n <META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n\n <META NAME="GENERATOR" CONTENT="Mozilla/4.04 [en] (Win95; I) [Netscape]">\n\n <TITLE>J', '', text)
  text = re.sub("\\n", "", text)
  text = re.sub("||", "", text)
  text = text.replace("||", "")
  return text

In [ ]:
dt_n_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/length_comp_news.json", lines=True)
dt_n_sec = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/unfiltered_news_v3.json", lines=True)
dt_n_thi = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Tweets.csv")
dt_n_fou = pd.read_csv("/content/gdrive/MyDrive/trained_models/data_for_trainig/Twitter_Data.csv")
dt_f_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/eng_literature.json", lines=True)

In [ ]:
books_data = []
with open('/content/gdrive/MyDrive/trained_models/data_for_trainig/Catch_Me_If_You_Can.txt') as f:
  lines = f.read()
while lines:
  books_data.append({"piece_of_text": re.sub("||", "", preproc(lines[:150])), "headline":"Catch_Me_If_You_Can"})
      # Set the contents to everything after the first 256
  lines = lines[150:]

In [ ]:
with open('/content/gdrive/MyDrive/trained_models/data_for_trainig/The_Eighth_Sister_Victoria_Heward.txt') as f:
  lines = f.read()
while lines:
  books_data.append({"piece_of_text": re.sub("||", "", preproc(lines[:150])), "headline":"The_Eighth_Sister_Victoria_Heward"})
      # Set the contents to everything after the first 256
  lines = lines[150:]

In [ ]:
with open('/content/gdrive/MyDrive/trained_models/data_for_trainig/London_John_Escott.txt') as f:
  lines = f.read()
while lines:
  books_data.append({"piece_of_text": re.sub("||", "", preproc(lines[:150])), "headline":"London_John_Escott"})
      # Set the contents to everything after the first 256
  lines = lines[150:]

In [ ]:
with open('/content/gdrive/MyDrive/trained_models/data_for_trainig/Hotel_Arthur_Hailey.txt') as f:
  lines = f.read()
while lines:
  books_data.append({"piece_of_text": re.sub("||", "", preproc(lines[:150])), "headline":"Hotel_Arthur_Hailey"})
      # Set the contents to everything after the first 256
  lines = lines[150:]

In [ ]:
with open('/content/gdrive/MyDrive/trained_models/data_for_trainig/UFOs_Helen_Brooke.txt') as f:
  lines = f.read()
while lines:
  books_data.append({"piece_of_text": re.sub("||", "", preproc(lines[:150])), "headline":"UFOs_Helen_Brooke"})
      # Set the contents to everything after the first 256
  lines = lines[150:]

In [ ]:
pieces_of_texts = []
for text in tqdm(dt_f_one.to_dict(orient="records"), total=len(dt_f_one["text"])):
  textt = preproc(text["text"])
  while textt:
    pieces_of_texts.append({"piece_of_text": re.sub("||", "", textt[:150]), "bookname": text["bookname"]})
        # Set the contents to everything after the first 256
    textt = textt[150:]

In [ ]:
pieces_of_news = []
for text in tqdm(dt_n_one.to_dict(orient="records"), total=len(dt_n_one["news_preproc"])):
  textt = preproc(text["news_preproc"])
  while textt:
    pieces_of_news.append({"piece_of_text": re.sub("||", "", textt[:150]), "headline": text["headline"]})
        # Set the contents to everything after the first 256
    textt = textt[150:]

In [ ]:
for text in tqdm(dt_n_thi.to_dict(orient="records"), total=len(dt_n_thi["text"])):
  try:
    textt = preproc(text["text"])
    while textt:
      pieces_of_news.append({"piece_of_text": re.sub("||", "", textt[:150]), "headline": "Tweets"})
          # Set the contents to everything after the first 256
      textt = textt[150:]
  except TypeError as tpe:
    continue

In [ ]:
for text in tqdm(dt_n_fou.to_dict(orient="records"), total=len(dt_n_fou["clean_text"])):
  try:
    textt = preproc(text["clean_text"])
    while textt:
      pieces_of_news.append({"piece_of_text": re.sub("||", "", textt[:150]), "headline": "Tweets"})
          # Set the contents to everything after the first 256
      textt = textt[150:]
  except TypeError as tpe:
    continue

In [ ]:
for text in tqdm(dt_n_sec.to_dict(orient="records"), total=len(dt_n_sec["short_description"])):
  textt = preproc(text["short_description"])
  while textt:
    pieces_of_news.append({"piece_of_text": re.sub("||", "", textt[:150]), "headline": text["headline"]})
        # Set the contents to everything after the first 256
    textt = textt[150:]

In [ ]:
pieces_of_data = [i["piece_of_text"] for i in pieces_of_texts] + [i["piece_of_text"] for i in pieces_of_news] + [i["headline"] for i in pieces_of_news] + [i["piece_of_text"] for i in books_data]

In [ ]:
dt = pd.DataFrame(ms_razbor).groupby(["count_word_net"]).head(15)

In [ ]:
results = []
contexts_d = []
for item in tqdm(dt.to_dict(orient="records"), total = len(dt["punchline"])):
  word = item["lemma"]
  contexts = []
  for t in pieces_of_data:
    if word != " " or word != "." or word != ",":
      if str(" " + word) in t or str(word + " ") in t:
        contexts.append(t)
      else:
        continue
    else:
      continue
  results.append({"punchline": item["punchline"], "word": item["word"],  "contexts_count": len(contexts)})
  contexts_d.append({"punchline": item["punchline"], "count_word_net": item["count_word_net"], item["word"]: contexts})

In [ ]:
contexted_data = pd.DataFrame(results).merge(dt, how="inner", on=["punchline", "word"])
contexted_data_n = contexted_data[contexted_data["pos"]=="NOUN"]

##### Vectorizing data

In [ ]:
from transformers import BertModel, BertTokenizer
bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )

In [ ]:
def bert_embed(sent):
  text = sent.lower().replace('"', '') # убираем лишние кавычки и приводим к нижнему регистру
  marked_text = "[CLS] " + text + " [SEP]" # добавляем начальные метки, указываем границы анализируемого предложения

  # делим предложения на токены
  tokenized_text = tokenizer.tokenize(marked_text)
  encodings = tokenizer(marked_text) # получаем начальные  представления

  # Сопоставляем токены с их словарными индексами.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Посмотреть на токены и индексы.
  #for tup in zip(tokenized_text, indexed_tokens):
  #    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)

      # Оценка модели вернет различное количество объектов в зависимости от того, как она была настроена в предыдущем вызове `from_pre trained`.
      # В этом случае, поскольку мы установили ' output_hidden_states = True, третьим элементом будут скрытые состояния из всех слоев.
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2] # берем скрытые состояния

  token_embeddings = torch.stack(hidden_states, dim=0) # предобрабатываем полученные скрытые состояния: объединяем последовательность тензоров вдоль новой новой размерности
  token_embeddings = torch.squeeze(token_embeddings, dim=1) # Возвращает новый тензор, где удалена часть входных данных с размером 1.
  token_embeddings = token_embeddings.permute(1,0,2) # перестановка данных для легкого изменения размеров тензора.

  token_embeddings.size() # векторные представления имеют размерность 22  x 768
  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers. # берем векторные представления с последних 4 слоев и суммируем их.
      sum_vec = torch.sum(token[-4:], dim=0)

      # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)

  return tokenized_text, token_vecs_sum

In [ ]:
def get_word_vec(tokenized_text, token_vecs_sum, text):
  text = text.lower().replace('"', '')
  pretok_word = ""
  ind = []
  vec = []
  tok_vec = []
  for i,tok in enumerate(tokenized_text):
    try:
      if tok in text.split(' ')[:i+3]:
        tok_vec.append({"word": tok, "vector": token_vecs_sum[i]}) # вектор привязываем к индексу, чтоб можно было достать
      elif tok not in text.split(' ')[:i+3]:
        #print(i, tok)
        if tok == '[CLS]' or tok == '[SEP]':
          continue
        elif tok.startswith("##"):
          pretok_word += tok[2:]
          ind.append(i)
          ind = list(set(ind))
          ind.sort()
          vec.append(token_vecs_sum[i])
        else:
          pretok_word += " " + tok
          ind.append(i)
          vec.append(token_vecs_sum[i])
        #print(pretok_word, ind,)
        comparison = list(set([pretok_word[1:].replace(' ', '')]) & set(text.split(' ')[:i+3]))
        #try:
        if len(comparison) > 0:
          word_string = "".join(tokenized_text[i] for i in ind).replace('#', '')
          tok_vec.append({"word": word_string, "vector":sum(vec)/ len(vec)})    #######достаем по последнем индексу списка
        elif len(comparison) == 0:
          continue
      comparison = []
      ind = []
      vec = []
      pretok_word = ""
      text = text.split(' ', 1)[1]
      #print(j_text)
    except IndexError as ie:
      pass

      return tok_vec

In [ ]:
missed_text = []
contexts_embed = [] # для сохранения векторных представления
words_embed= []
for diction in tqdm(contexts_d, total=len(contexts_d)):
  for k, list_cont in diction.items():
    if k != "punchline" and k!= "count_word_net":
      tokenized_text, token_vecs_sum_w = bert_embed(sent=diction["punchline"])
      tok_vec = get_word_vec(tokenized_text= tokenized_text, token_vecs_sum = token_vecs_sum_w, text=diction["punchline"])
      word_vec =[i["vector"] for i in tok_vec if i["word"] ==  k][0]
      words_embed.append({"punchline": diction["punchline"],
                "word":k,
                "count_m": diction["count_word_net"],
                "word_vec": word_vec})
      for cont in list_cont[:500]:
        _, token_vecs_sum = bert_embed(sent =cont)
        b_d = {"punchline": diction["punchline"],
                "word":k,
                "context": cont,
              "bert_cont": sum(token_vecs_sum)/len(token_vecs_sum)} # делаем векторное представление шутки через среднее арифметическое векторных представлений BERT
        contexts_embed.append(b_d)
      else:
        continue
    else:
      continue

In [ ]:
#### сохранение данных

import pickle
import torch

a = torch.rand(3,4,5)

# save
with open('/content/gdrive/MyDrive/trained_models/filename_words_embed_500_fin.pickle', 'wb') as handle:
    pickle.dump(words_embed, handle)

with open('/content/gdrive/MyDrive/trained_models/filename_context_embed_500_fin.pickle', 'wb') as handle:
    pickle.dump(contexts_embed, handle)

# open
#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

##### Results
To assess the impact of a polysemic word on the overall text vector, the cosine distances between the polysemic word and a joke with deleted polysemic word, and a non-polysemic word and a joke with deleted non-polysemic word were measured. The distance between cluster centroids, and the cosine distances of the two types were analysed using Student’s t distribution. For polysemic words, p-value=0.003, for non-polysemic words pvalue<10^-6.  We can see that if a joke loses either a polysemic or a non-polysemic word, its vectorisation changes. Furthermore, in these two cases it changes differently.

In [ ]:
!pip install transformers==4.37.2

In [ ]:
#Загрузка библиотек
import string
import re
import numpy as np
from numpy import array, argmax, random, take
from tqdm import tqdm
import pandas as pd
from keras.models import load_model
from keras import optimizers
import json
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
import spacy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pickle

with open('/content/gdrive/MyDrive/trained_models/filename_words_embed_500_fin.pickle', 'rb') as handle:
    words_embed = pickle.load(handle)

with open('/content/gdrive/MyDrive/trained_models/filename_context_embed_500_fin.pickle', 'rb') as handle:
    contexts_embed = pickle.load(handle)

In [ ]:
import heapq
from sklearn.cluster import AffinityPropagation, MeanShift
from scipy.spatial import distance

all_res = []
count_clusters = []
for item in tqdm(words_embed, total=len(words_embed)):
  vecs = []
  for it in contexts_embed:
    if item["word"] == it["word"]:
      vecs.append(it["bert_cont"])
  clustering = MeanShift(bandwidth = 30).fit([item["word_vec"].numpy()] + [i.numpy() for i in vecs]) #######  проблема в снижении размерности???????
  count_clusters.append({"word": item["word"],"count_cluster":len(set(clustering.labels_)), "count_contexts": len(vecs) })
  centroids = clustering.cluster_centers_
  distances = np.empty((0,len(centroids)), float)
  # getting points and distances
  for center_elem in centroids:
      for i in centroids:
      # cdist is used to calculate the distance between center and other points
        distances = np.append(distances, distance.cdist([center_elem],[i], 'cosine'))
  try:
    distances =np.delete(distances, np.where(distances==0.0))
    distances =np.delete(distances, np.where(distances==1.1102230246251565e-16))
    if min(distances) < 0.00000000001:
      all_res.append({"punchline":item["punchline"], "word": item["word"], "count_word_net": item["count_m"], "min_cos_dist":0})
    else:
      all_res.append({"punchline":item["punchline"],"word": item["word"], "count_word_net": item["count_m"], "min_cos_dist":min(distances)})
  except ValueError as ve:
    all_res.append({"punchline":item["punchline"], "word": item["word"], "min_cos_dist":0})

Количество кластеров у каждого слова

In [ ]:
clusters_df = pd.DataFrame(count_clusters)
clusters_df['count_contexts'] = [1 if i==0 else i for i in clusters_df['count_contexts']]
set(clusters_df["count_cluster"])

In [ ]:
stop_symbols = [',', ".", ";", ":", "!", "'", '"', "/", '\\', '-', '–' '(', ")", '=', '+', "—", '’', '#', "?", "...", " "]
test_df = pd.read_csv("/content/gdrive/MyDrive/trained_models/exp_triggers_test_df_bigdata_38k.csv")
test_df["label_res"] =  test_df["label"].apply(lambda x: "joke" if x == 0 else "non-joke")
test_df["text"] =  [ i.replace('...', '.').replace("  ", ' ') for  i in test_df["text"]]
test_df["text_unpreproc"] = test_df["text_unpreproc"].fillna(test_df["text"])
test_df["text_unpreproc"] = test_df["text_unpreproc"].apply(lambda i: str(i + " .") if i[-1] not in stop_symbols else i)
test_df["text"] =  test_df["text"].apply(lambda i: None if i[-1] not in stop_symbols else i)
test_df["text"] = test_df["text"].fillna(test_df["text_unpreproc"])
test_df["text"]  = [i.lower().replace('.', ' .').replace('!', ' !').replace('?', ' ?') for i in test_df["text"]]
#test_df["text"].apply(lambda i: str(i + ".") if i[-1] not in stop_symbols else i)
test_df= test_df.groupby(["label"]).head(100)
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df = test_df[test_df["label"] == 0].reset_index(drop=True).reindex()

In [ ]:
new_df = test_df.merge(pd.DataFrame(all_res), on=["punchline"], how="inner")
new_df["joke_out"] = [str(i["setup"] + ' ' +i["punchline"].replace(i["word"], "")) for i in new_df.to_dict(orient="records")]
set(new_df["min_cos_dist"])

In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertTokenizer

bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )

In [ ]:
def bert_embed(sent):
  text = sent.lower().replace('"', '') # убираем лишние кавычки и приводим к нижнему регистру
  marked_text = "[CLS] " + text + " [SEP]" # добавляем начальные метки, указываем границы анализируемого предложения

  # делим предложения на токены
  tokenized_text = tokenizer.tokenize(marked_text)
  encodings = tokenizer(marked_text) # получаем начальные  представления

  # Сопоставляем токены с их словарными индексами.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Посмотреть на токены и индексы.
  #for tup in zip(tokenized_text, indexed_tokens):
  #    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)

      # Оценка модели вернет различное количество объектов в зависимости от того, как она была настроена в предыдущем вызове `from_pre trained`.
      # В этом случае, поскольку мы установили ' output_hidden_states = True, третьим элементом будут скрытые состояния из всех слоев.
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2] # берем скрытые состояния

  token_embeddings = torch.stack(hidden_states, dim=0) # предобрабатываем полученные скрытые состояния: объединяем последовательность тензоров вдоль новой новой размерности
  token_embeddings = torch.squeeze(token_embeddings, dim=1) # Возвращает новый тензор, где удалена часть входных данных с размером 1.
  token_embeddings = token_embeddings.permute(1,0,2) # перестановка данных для легкого изменения размеров тензора.

  token_embeddings.size() # векторные представления имеют размерность 22  x 768
  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers. # берем векторные представления с последних 4 слоев и суммируем их.
      sum_vec = torch.sum(token[-4:], dim=0)

      # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)

  vector = sum(token_vecs_sum) / len(token_vecs_sum)
  return vector

In [ ]:
from numpy import dot
from numpy.linalg import norm

#cos_sim = dot(a, b)/(norm(a)*norm(b))
missed = []
cosines = []
for diction in new_df.to_dict(orient="records"):
  word_vec = [i["word_vec"] for i in words_embed if i["word"]== diction["word"]]
  vector = bert_embed(sent =diction["joke_out"])
    #cos_sim = dot(vector, word_vec["word_vec"])/(norm(vector)*norm(word_vec["word_vec"]))
  cos_sim =distance.cdist([word_vec[0]], [vector], "cosine")
  b_d = {"joke_out": diction["joke_out"], "word": diction["word"] ,
          "cos_word_v_changed_v": cos_sim} # делаем векторное представление шутки через среднее арифметическое векторных представлений BERT
  cosines.append(b_d)




In [ ]:
new_df = new_df.merge(pd.DataFrame(cosines), on=[ "joke_out","word"], how="inner")

In [ ]:
len(new_df[new_df["count_word_net"] == 2])

In [ ]:
from scipy.stats import ttest_ind
import numpy as np
import numpy.ma as ma
print(ttest_ind(a=[i for i in new_df[new_df["count_word_net"] == 1]["min_cos_dist"]], b=[i[0][0] for  i in new_df[new_df["count_word_net"] == 1]["cos_word_v_changed_v"]]) )

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xticks([0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plot = sns.histplot(data=new_df[new_df["count_word_net"] == 2], x=[i[0][0] for  i in new_df[new_df["count_word_net"] == 2]["cos_word_v_changed_v"]],
                    ax =ax, color="red",  bins=10, stat='density', binrange=(0.3, 0.8))
plot = sns.histplot(data=new_df[new_df["count_word_net"] == 1], x=[i[0][0] for  i in new_df[new_df["count_word_net"] == 1]["cos_word_v_changed_v"]], ax =ax,
                    bins=10, stat='density', binrange=(0.3, 0.8))

#### Experiment: syntactic probing

we conducted an experiment on syntactic probing of a “bert-base-uncased” model. In the experiment, jokes’ setup and punchline were swapped, and then the jokes were submitted as input to a classifier to categorize them as humorous or non-humorous. The main corpus comprised 2,500 unchanged humorous texts, 2,500 humorous texts where setup and punchline were swapped, and 500 non-humorous texts.

##### Loading and processing data

In [ ]:
!pip install transformers==4.37.2

In [ ]:
#Загрузка библиотек
import string
import re
import numpy as np
from numpy import array, argmax, random, take
from tqdm import tqdm
from keras.models import load_model
from keras import optimizers
import json
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import torch
import os
import pandas as pd
from transformers import BertModel, BertPreTrainedModel
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch import nn
from torch.nn import BCEWithLogitsLoss
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from nltk.corpus import stopwords
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

!ls /content/gdrive/MyDrive/trained_models


In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertTokenizer

bert_model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )

In [ ]:
dt_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/jokes_exp_triggers_24k.json", lines= True)
dt_reddit_jokes = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/reddit_jokes.json")

In [ ]:
jokes = [str( i + " "+ u ).lower() for i,u in zip(dt_reddit_jokes["title"][:62000], dt_reddit_jokes["body"][:62000]) if len(str(i+ " " + u)) <= 256]

In [ ]:
sekoj = [str( u + " "+ i).lower() for i,u in zip(dt_reddit_jokes["title"][:62000], dt_reddit_jokes["body"][:62000]) if len(str(i+ " " + u)) <= 256]

In [ ]:
jokes = jokes[:5000]
sekoj = sekoj[:5000]

In [ ]:
dt_n_one = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/length_comp_news.json", lines=True)
dt_n_sec = pd.read_json("/content/gdrive/MyDrive/trained_models/data_for_trainig/unfiltered_news_v3.json", lines=True)

In [ ]:
news = [i for i in dt_n_one["headline"] if len(i) <= 256] + [i for i in dt_n_one["news_preproc"] if len(i) <= 256]

In [ ]:
news = news[:500]
len(news)

In [ ]:
def bert_embed(sent):
  text = sent.lower().replace('"', '')
  marked_text = "[CLS] " + text + " [SEP]"


  tokenized_text = tokenizer.tokenize(marked_text)
  encodings = tokenizer(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)



  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

      outputs = bert_model(tokens_tensor, segments_tensors)

      hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  token_embeddings.size()
  token_vecs_sum = []

  for token in token_embeddings:

      sum_vec = torch.sum(token[-4:], dim=0)

      token_vecs_sum.append(sum_vec)

  vector = sum(token_vecs_sum) / len(token_vecs_sum)
  return vector

In [ ]:
from numpy import dot
from numpy.linalg import norm

#cos_sim = dot(a, b)/(norm(a)*norm(b))
missed = []
old_embedded = []
embedded = []
for i, joke in enumerate(jokes[:2500]):
  vector = bert_embed(sent = joke)
  embedded.append({"id":"joke_{}".format(i), "text": joke, "label":0, "vector": vector})
for i, joke in enumerate(sekoj[:2500]):
  vector = bert_embed(sent = joke)
  old_embedded.append({"id":"joke_{}".format(i), "text": joke, "label":0, "vector": vector})
for i,new in enumerate(news):
  vector = bert_embed(sent = new)
  embedded.append({"id":"new_{}".format(i),"text": new, "label":2, "vector": vector})


In [ ]:
batch_size= 32
max_length =256
class BertEmbedLayer(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentences: Array of input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentences,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentences = sentences
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentences))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentences) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentences = self.sentences[indexes]

        # With BERT tokenizer's batch_encode_plus batch of text data are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentences.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
from transformers import TFBertModel
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/trained_models/model_june_bigdata_38k_torch_no_grad.h5', custom_objects={'Custom>TFBertMainLayer':TFBertModel.from_pretrained("bert-base-uncased")})

In [ ]:
labels = ["joke", 'non-joke']
first_results = []
for item in tqdm(embedded, total= len(embedded)):
  sentences = np.array([item["text"]])
  test_data = BertEmbedLayer(
      sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
  )

  proba = new_model.predict(test_data[0])[0]
  idx = np.argmax(proba)
  proba = float('%.2f' % (proba[idx])) #%
  pred = labels[idx]
  first_results.append({"id": item["id"], "text": item["text"], "label": item["label"],"model_res_digit_fm": idx, "model_res_fm": pred, "model_proba_fm": proba})

In [ ]:
labels = ["joke", 'non-joke']
sec_results = []
i = 0
for item in tqdm(old_embedded[:2500], total= len(old_embedded[:2500])):
  sentences = np.array([item["text"]])
  test_data = BertEmbedLayer(
      sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
  )

  proba = new_model.predict(test_data[0])[0]
  idx = np.argmax(proba)
  proba = float('%.2f' % (proba[idx])) #%
  pred = labels[idx]
  sec_results.append({"id":  item["id"],"text": item["text"], "label": item["label"],"model_res_digit_fm": idx, "model_res_fm": pred, "model_proba_fm": proba})

In [ ]:
new_cetroid = np.mean([ i["vector"] for i in embedded[2500:3000]])
new_cetroid

In [ ]:
dt_jokes = pd.DataFrame(embedded).merge(pd.DataFrame(first_results), on=["id","text", "label"], how="inner")
dt_jokes = dt_jokes[dt_jokes["label"] != 2].reset_index(drop=True).reindex()
dt_sekoj = pd.DataFrame(old_embedded).merge(pd.DataFrame(sec_results), on=["id","text", "label"], how="inner")
dt_sekoj["id"] = "punchline_setup"

In [ ]:
dt_jokes["distance"] =new_cetroid - dt_jokes["vector"]
dt_sekoj["distance"] = new_cetroid - dt_sekoj["vector"]

In [ ]:
dt_jokes["distance"]  = [np.linalg.norm(i) for i in dt_jokes["distance"]]
dt_sekoj["distance"]  = [np.linalg.norm(i) for i in dt_sekoj["distance"]]

In [ ]:
both = pd.concat([dt_jokes, dt_sekoj])

In [ ]:
# T-test
from scipy.stats import ttest_ind
import numpy as np
import numpy.ma as ma
print(ttest_ind(a=np.array([ i for i in dt_jokes["distance"]]), b=np.array([i for i in dt_sekoj["distance"]])))

In [ ]:
len(both[(both["model_res_digit_fm"] == 1) & (both["id"].str.contains("punchline_setup")== True)])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xticks([0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plot = sns.histplot(data=both[(both["model_res_digit_fm"] == 0) & (both["id"].str.contains("joke")== True)], x="distance", ax =ax, color="red") # для шуток без перестановки,
plot = sns.histplot(data=both[(both["model_res_digit_fm"] == 0) & (both["id"].str.contains("punchline_setup")==True)], x="distance", ax =ax)
plot.set(xlabel='Distance between news centroid and "setup-punchline" jokes and between news centroid and "punchline-setup" jokes: unchanged label', ylabel='Count')

In [ ]:
# T-test
from scipy.stats import ttest_ind
import numpy as np
import numpy.ma as ma
print(ttest_ind(a=np.array([ i for i in both[(both["model_res_digit_fm"] == 0) & (both["id"].str.contains("joke")== True)]["distance"]]), b=np.array([i for i in both[(both["model_res_digit_fm"] == 0) & (both["id"].str.contains("punchline_setup")==True)]["distance"]])))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xticks([0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plot = sns.histplot(data=both[(both["model_res_digit_fm"] == 1) & (both["id"].str.contains("joke")== True)], x="distance", ax =ax, color="red", bins= 15) # для шуток без перестановки,
plot = sns.histplot(data=both[(both["model_res_digit_fm"] == 1) & (both["id"].str.contains("punchline_setup")==True)], x="distance", ax =ax, bins=15)
plot.set(xlabel='Distance between news centroid and "setup-punchline" jokes and between news centroid and "punchline-setup" jokes:changed label', ylabel='Count')

In [ ]:
# T-test
from scipy.stats import ttest_ind
import numpy as np
import numpy.ma as ma
print(ttest_ind(a=np.array([ i for i in both[(both["model_res_digit_fm"] == 1) & (both["id"].str.contains("joke")== True)]["distance"]]), b=np.array([i for i in both[(both["model_res_digit_fm"] == 1) & (both["id"].str.contains("punchline_setup")==True)]["distance"]])))

##### UMAP processing


 The data was vectorised by the “bert-base-uncased” model and used as input for the classifier. Text embeddings were processed using the UMAP algorithm. Actually humorous texts recognised as such by the classifier are marked in red. Actually humorous texts recognised as non-humorous are marked in green. News texts are marked in blue.

In [ ]:
!pip install umap-learn

In [ ]:
#UMAP-LEARN
import umap

umap_text = umap.UMAP(n_components=2, random_state=100)
umap_reduced = umap_text.fit_transform([i["vector"]for i in embedded] + [i["vector"]for i in old_embedded[:2500]])

In [ ]:

####Картинка UMAP
fig, ax = plt.subplots(1,1, figsize= (13, 15))
#ax = fig.add_subplot(111, projection='3d')
colors = [i["label"] for i in embedded] + [i["model_res_digit_fm"] for i in sec_results]
ax.scatter(umap_reduced[:, 0], umap_reduced[:, 1], c=["red" if i == 0 else "blue" if i==2 else "green" for i in colors], ) # train_b[:, 0], train_b[:, 1], c=list(k_means_bert) - нужно заменить, чтобы получить виз-цию результатов для Берта.


#plt.xlim(xmin=0, xmax=30)
# only show 100 - 1600 on y-axis.
#plt.ylim(ymin=0, ymax=30)

ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
plt.title("График распределения шуток и не-шуток в векторном пространстве")
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

plt.show()
plt.savefig(fname = "data_200_color.png")

#### Experiments with eliminating random words from text

In this experiment, we eliminated certain text fragments identified as potentially significant semantic concepts from all texts. This approach is more relevant for jokes than for news or fiction, since the latter tend to be ordinary non-humorous texts from the start (Raskin V., 1984). The results are presented for a part of the validation corpus comprising 100 jokes. Following the initial assessment by the model, the validation dataset was subjected to a series of experimental modifications to find out after which changes jokes cease to be humorous and turn into ordinary texts. Text fragments were eliminated on the basis of their potential impact on the semantic meaning in different parts of the texts, especially humorous ones. The working unit was a token: a word or punctuation mark. In the course of designing the experiments, tokens’ positions relative to the sequence structure, lexical expression, and syntactic relations were taken into account

##### Load main dataset

In [ ]:
!pip install transformers

In [ ]:
# Load, explore and plot data

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
from sklearn.model_selection import train_test_split# Text pre-processing
import tensorflow as tf
#tf.config.run_functions_eagerly(True)
from transformers import BertTokenizer,TFBertModel
from sklearn.pipeline import Pipeline

In [ ]:
#!pip install --upgrade scikit-learn

In [ ]:

import string
import re
import numpy as np
from numpy import array, argmax, random, take
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import json
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import torch
import os
import pandas as pd
from transformers import BertModel, BertPreTrainedModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.model_selection import train_test_split
import random
import re
import matplotlib
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch import nn
from torch.nn import BCEWithLogitsLoss
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from nltk.corpus import stopwords
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import math


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from nltk import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
max_length = 256  # Maximum length of input sentence to the model.
batch_size = 32

# Labels in our dataset.
#labels_sec = list(set(dfnj_sec["label"]))
#stop_w = stopwords.words('english')


In [ ]:
class BertEmbedLayer(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentences: Array of input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentences,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentences = sentences
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentences))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentences) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentences = self.sentences[indexes]

        # With BERT tokenizer's batch_encode_plus batch of text data are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentences.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

!ls /content/gdrive/MyDrive/trained_models

In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/trained_models/model_june_bigdata_38k_torch_no_grad.h5', custom_objects={'TFBertModel':TFBertModel.from_pretrained("bert-base-uncased")})

In [ ]:
stop_symbols = [',', ".", ";", ":", "!", "'", '"', "/", '\\', '-', '–' '(', ")", '=', '+', "—", '’', '#', "?", "...", " "]
test_df = pd.read_csv("/content/gdrive/MyDrive/trained_models/exp_triggers_test_df_bigdata_38k.csv")
test_df["label_res"] =  test_df["label"].apply(lambda x: "joke" if x == 0 else "non-joke")
test_df["text"] =  [ i.replace('...', '.').replace("  ", ' ') for  i in test_df["text"]]
test_df["text_unpreproc"] = test_df["text_unpreproc"].fillna(test_df["text"])
test_df["text_unpreproc"] = test_df["text_unpreproc"].apply(lambda i: str(i + " .") if i[-1] not in stop_symbols else i)
test_df["text"] =  test_df["text"].apply(lambda i: None if i[-1] not in stop_symbols else i)
test_df["text"] = test_df["text"].fillna(test_df["text_unpreproc"])
test_df["text"]  = [i.lower().replace('.', ' .').replace('!', ' !').replace('?', ' ?') for i in test_df["text"]]
#test_df["text"].apply(lambda i: str(i + ".") if i[-1] not in stop_symbols else i)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=2)
test_df= test_df.groupby(["label"]).head(100)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
test_data = BertEmbedLayer(
    test_df["text"].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
labels = ["joke", 'non-joke']
first_results = []
for item in tqdm(test_df.to_dict(orient="records"), total= len(test_df.to_dict(orient="records"))):
  sentences = np.array([item["text"]])
  test_data = BertEmbedLayer(
      sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
  )

  proba = new_model.predict(test_data[0])[0]
  idx = np.argmax(proba)
  proba = float('%.2f' % (proba[idx])) #%
  pred = labels[idx]
  first_results.append({"id":item["id"], "text": item["text"], "text_unpreproc": item["text_unpreproc"], "label": item["label"], "label_res": item["label_res"], "model_res_digit_fm": idx, "model_res_fm": pred, "model_proba_fm": proba})

In [ ]:
df_fr = pd.DataFrame(first_results)
df_fr["id"] = ["sfj5fkjf_{}_{}".format(i,l) for i, l in enumerate(df_fr["label_res"])]

In [ ]:
classes = [0, 1]
con_mat =tf.math.confusion_matrix(labels=df_fr["label"], predictions=df_fr["model_res_digit_fm"]).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = labels,
                     columns = labels)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
df_fr.to_csv("/content/gdrive/MyDrive/trained_models/exp_triggers_test_df_bigdata_38k_df_fr.csv")

In [ ]:
df_fr = pd.read_csv("/content/gdrive/MyDrive/trained_models/exp_triggers_test_df_bigdata_38k_df_fr.csv")

##### Words' elimination

###### Eliminating tokens at the end of the text.

In [ ]:
# Тест 1: выкидываем с конца

last_word_l = []
last_two_words_l = []
last_three_words_l = []
for item in tqdm(df_fr.to_dict(orient="records"), total = len(df_fr.to_dict(orient="records"))):
  tokenized_text = nltk.word_tokenize(item["text"].lower())
  tokenized_text = [w for w in tokenized_text]
  last_word = tokenized_text[-2] # меняем вместо единицы на -2, чтобы у нас оставались знаки препинания
  last_two_words = " ".join([tokenized_text[len(tokenized_text) - 3], tokenized_text[-2]])
  last_three_words =  " ".join([tokenized_text[len(tokenized_text) - 4], tokenized_text[len(tokenized_text) - 3], tokenized_text[-2]])
  last_word_l.append({
                      "text":item["text"],
                      "label": item["label"],
                      "strip_last_word": " ".join(tokenized_text).replace(last_word, '')})
  last_two_words_l.append({"text":item["text"],
                      "label": item["label"],
                      "strip_last_two_words": " ".join(tokenized_text).replace(last_two_words, '')})
  last_three_words_l.append({"text":item["text"],
                      "label": item["label"],
                      "strip_last_three_words": " ".join(tokenized_text).replace(last_three_words, '')})

In [ ]:
df_fr_last = df_fr.merge(pd.DataFrame(last_word_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(last_two_words_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(last_three_words_l), on = ["text", "label"], how='inner')

In [ ]:
def get_model_proba_strip_words(count_w):
  cutoff_results = []
  for item in tqdm(df_fr_last.to_dict(orient="records"), total= len(df_fr_last.to_dict(orient="records"))):
    sentences = np.array([item[count_w]])
    test_data = BertEmbedLayer(
        sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = new_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = proba[idx]
    pred = labels[idx]
    cutoff_results.append({"id": item["id"],
                           "text": item["text"],
                            "strip_words": item[count_w],
                            "model_res_digit": idx,
                            "model_res": pred,
                            "model_proba": round(proba, 4),
                           "type_cut_off_words": count_w})
  return cutoff_results

In [ ]:
list_strip_words = ["strip_last_word", "strip_last_two_words", "strip_last_three_words"]
last_three_words = get_model_proba_strip_words(count_w="strip_last_three_words")
last_two_words = get_model_proba_strip_words(count_w="strip_last_two_words")
last_one_word = get_model_proba_strip_words(count_w="strip_last_word")

In [ ]:
all_str_df = pd.concat([pd.DataFrame(last_three_words), pd.DataFrame(last_two_words), pd.DataFrame(last_one_word)])

In [ ]:
all_str_df =all_str_df.merge(all_str_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

all_str_df = all_str_df.rename(columns={"model_proba": "worst_proba"})

In [ ]:
del df_fr_last["strip_last_word"]
del df_fr_last["strip_last_two_words"]
del df_fr_last["strip_last_three_words"]

df_fr_last = df_fr_last.merge(all_str_df, on=["id", "text"], how='inner')

In [ ]:
df_fr_last = df_fr_last.sort_values(by=["label_res"], ascending= False)

###### Eliminating at the beginning of the text.

In [ ]:



first_word_l = []
first_two_words_l = []
first_three_words_l = []
for item in tqdm(df_fr.to_dict(orient="records"), total = len(df_fr.to_dict(orient="records"))):
  try:
    tokenized_text = nltk.word_tokenize(item["text"].lower())
    tokenized_text = [w for w in tokenized_text if w not in stop_symbols]
    first_word = tokenized_text[0]
    first_two_words = " ".join([tokenized_text[0], tokenized_text[1]])
    first_three_words =  " ".join([tokenized_text[0], tokenized_text[1], tokenized_text[2]])
  except IndexError as ie:
    first_three_words =  " ".join([tokenized_text[0], tokenized_text[1]])
  first_word_l.append({
                      "text":item["text"],
                      "label": item["label"],
                      "strip_first_word": " ".join(tokenized_text).replace(first_word, '')})
  first_two_words_l.append({"text":item["text"],
                      "label": item["label"],
                      "strip_first_two_words": " ".join(tokenized_text).replace(first_two_words, '')})
  first_three_words_l.append({"text":item["text"],
                      "label": item["label"],
                      "strip_first_three_words": " ".join(tokenized_text).replace(first_three_words, '')})

In [ ]:
df_fr_fir = df_fr.merge(pd.DataFrame(first_word_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(first_two_words_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(first_three_words_l), on = ["text", "label"], how='inner')

In [ ]:

def get_model_proba_strip_words(count_w):
  cutoff_results = []
  for item in tqdm(df_fr_fir.to_dict(orient="records"), total= len(df_fr_fir.to_dict(orient="records"))):
    sentences = np.array([item[count_w]])
    test_data = BertEmbedLayer(
        sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = new_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = proba[idx]
    pred = labels[idx]
    cutoff_results.append({"id": item["id"],
                           "text": item["text"],
                            "strip_words": item[count_w],
                            "model_res_digit": idx,
                            "model_res": pred,
                            "model_proba": proba,
                           "count_cut_off_words": count_w})
  return cutoff_results

In [ ]:
list_strip_words = ["strip_first_word", "strip_first_two_words", "strip_first_three_words"]
first_three_words = get_model_proba_strip_words(count_w="strip_first_three_words")
first_two_words = get_model_proba_strip_words(count_w="strip_first_two_words")
first_one_word = get_model_proba_strip_words(count_w="strip_first_word")

In [ ]:
all_str_df = pd.concat([pd.DataFrame(first_three_words), pd.DataFrame(first_two_words), pd.DataFrame(first_one_word)])

In [ ]:
all_str_df =all_str_df.merge(all_str_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

all_str_df = all_str_df.rename(columns={"model_proba": "worst_proba"})

In [ ]:
del df_fr_fir["strip_first_word"]
del df_fr_fir["strip_first_two_words"]
del df_fr_fir["strip_first_three_words"]


df_fr_fir = df_fr_fir.merge(all_str_df, on=["id", "text"], how='inner')

In [ ]:
df_fr_fir = df_fr_fir.sort_values(by=["label_res"], ascending= False)

###### Eliminating in the middle of the text.

In [ ]:
middle_word_l = []
middle_two_words_l = []
middle_three_words_l = []
for item in tqdm(df_fr.to_dict(orient="records"), total = len(df_fr.to_dict(orient="records"))):
  try:
      tokenized_text = nltk.word_tokenize(item["text"].lower())
      tokenized_text = [w for w in tokenized_text if w not in stop_symbols]
      middle_word = tokenized_text[math.ceil(len(tokenized_text) / 2)]
      middle_two_words = " ".join([tokenized_text[math.ceil(len(tokenized_text) / 2)], tokenized_text[math.ceil(len(tokenized_text) / 2) + 1]])
      middle_three_words =  " ".join([tokenized_text[math.ceil(len(tokenized_text) / 2) - 1], tokenized_text[math.ceil(len(tokenized_text) / 2)], tokenized_text[math.ceil(len(tokenized_text) / 2) + 1]])
      middle_word_l.append({
                          "text":item["text"],
                          "label": item["label"],
                          "strip_middle_word": " ".join(tokenized_text).replace(middle_word, '')})
      middle_two_words_l.append({"text":item["text"],
                          "label": item["label"],
                          "strip_middle_two_words": " ".join(tokenized_text).replace(middle_two_words, '')})
      middle_three_words_l.append({"text":item["text"],
                          "label": item["label"],
                          "strip_middle_three_words": " ".join(tokenized_text).replace(middle_three_words, '')})
  except IndexError as ie:
    continue

In [ ]:
df_fr_mid = df_fr.merge(pd.DataFrame(middle_word_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(middle_two_words_l), on = ["text", "label"], how='inner').merge(pd.DataFrame(middle_three_words_l), on = ["text", "label"], how='inner')

In [ ]:
labels = ["joke", 'non-joke']
def get_model_proba_strip_words(count_w):
  cutoff_results = []
  for item in tqdm(df_fr_mid.to_dict(orient="records"), total= len(df_fr_mid.to_dict(orient="records"))):
    sentences = np.array([item[count_w]])
    test_data = BertEmbedLayer(
        sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = new_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = proba[idx]
    pred = labels[idx]
    cutoff_results.append({"id": item["id"],
                           "text": item["text"],
                            "strip_words": item[count_w],
                            "model_res_digit": idx,
                            "model_res": pred,
                            "model_proba": proba,
                           "count_cut_off_words": count_w})
  return cutoff_results

In [ ]:
list_strip_words = ["strip_middle_word", "strip_middle_two_words", "strip_middle_three_words"]
middle_three_words = get_model_proba_strip_words(count_w="strip_middle_three_words")
middle_two_words = get_model_proba_strip_words(count_w="strip_middle_two_words")
middle_one_word = get_model_proba_strip_words(count_w="strip_middle_word")

In [ ]:
all_str_df = pd.concat([pd.DataFrame(middle_three_words), pd.DataFrame(middle_two_words), pd.DataFrame(middle_one_word)])

In [ ]:
all_str_df =all_str_df.merge(all_str_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

all_str_df = all_str_df.rename(columns={"model_proba": "worst_proba"})


In [ ]:
del df_fr_mid["strip_middle_word"]
del df_fr_mid["strip_middle_two_words"]
del df_fr_mid["strip_middle_three_words"]

df_fr_mid = df_fr_mid.merge(all_str_df, on=["id", "text"], how='inner')
df_fr_mid = df_fr_mid.sort_values(by=["id", "text", "model_proba_fm", "worst_proba"], ascending=False)

In [ ]:
df_fr_mid = df_fr_mid.sort_values(by=["label_res"], ascending= False)

###### SpaCy Preprocessing

In [ ]:
!pip install en_core_web_sm
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls /content/gdrive/MyDrive/trained_models

In [ ]:
del test_df[test_df.columns[0]]

In [ ]:
ling_df = df_fr.merge(test_df, on=["text", "text_unpreproc", "label", "label_res"], how="inner")

In [ ]:
spacy_model = spacy.load("en_core_web_sm")

Основные функции

In [ ]:
def get_model_proba_setup_punch(setup, punchline):
  labels = ["joke", "non-joke"]
  cutoff_results = []
  for item in tqdm(ling_df.to_dict(orient="records"), total= len(ling_df.to_dict(orient="records"))):
    sentences = np.array([" ".join([item[setup], item[punchline]])])
    test_data = BertEmbedLayer(
        sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = new_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = proba[idx]
    pred = labels[idx]
    cutoff_results.append({"id": item["id"],
                           "text": item["text"],
                           "label": item["label"],
                           "label_res": item["label_res"],
                            "model_res_digit_fm": item["model_res_digit_fm"],
                            "model_res_fm": item["model_res_fm"],
                           "model_proba_fm": item["model_proba_fm"],
                            "text_out": " ".join([item[setup], item[punchline]]),
                           "type_text_out": "_".join([setup, punchline]),
                            "model_res_digit": idx,
                            "model_res": pred,
                            "model_proba": proba,
                           "text_unpreproc": item["text_unpreproc"]})
  return cutoff_results



In [ ]:
def get_model_proba_text(text):
  labels = ["joke", "non-joke"]
  cutoff_results = []
  for item in tqdm(ling_df.to_dict(orient="records"), total= len(ling_df.to_dict(orient="records"))): #ling_df
    sentences = np.array([item[text]])
    test_data = BertEmbedLayer(
        sentences, labels=labels, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = new_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = proba[idx]
    pred = labels[idx]
    cutoff_results.append({"id": item["id"],
                           "text": item["text"],
                           "label": item["label"],
                           "label_res": item["label_res"],
                            "model_res_digit_fm": item["model_res_digit_fm"],
                            "model_res_fm": item["model_res_fm"],
                           "model_proba_fm": item["model_proba_fm"],
                            "changed_text": item[text] , #text_out
                           "type_text_out": text,
                            "model_res_digit": idx,
                            "model_res": pred,
                            "model_proba": proba,
                           "text_unpreproc": item["text_unpreproc"]})
  return cutoff_results

###### Eliminating a single noun/proper noun/pronoun

In [ ]:
def clean_first_noun(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  tokenized_text = [token.text for token in doc]
  clean_text = []
  for i, token in enumerate(doc):
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.pos_ == 'NOUN' or  token.pos_ == 'PROPN':
        del tokenized_text[i]
        break
      elif token.pos_ != 'NOUN' or  token.pos_ != 'PROPN':
        continue
  clean_text = " ".join(tokenized_text)
  return clean_text

In [ ]:
ling_df_fr = ling_df

In [ ]:
ling_df_fr["setup_out"] = [clean_first_noun(s) for s in ling_df_fr["setup"]]
ling_df_fr["punchline_out"] = [clean_first_noun(s) for s in ling_df_fr["punchline"]]
ling_df_fr["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_fr["setup_out"], ling_df_fr["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_onen =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_onen = res_df_onen.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_onen = res_df_onen.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_onen.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_one_noun_1000.csv")

###### Eliminating all nouns/proper names/pronouns.

In [ ]:
ling_df_sec= ling_df

In [ ]:

def non_joke_division(news):
  jokes_missed = []
  news = str(news)
  setup = ''
  punchline= ''
  if "," in news:
      joke_division = news.rsplit(",", 1)
      setup =  joke_division[0]
      punchline = joke_division[1]
  elif "?" in news:
      joke_division = news.rsplit("?", 1)
      setup =  joke_division[0]
      punchline = joke_division[1]
  elif "." in news: #and news.rsplit(".", 1)[1] != '': #### ПРОБЛЕМА С ДВУМЯ ПРЕДЛОЖЕНИЯМИ
      if news.count('.') > 1:
          joke_division = news.rsplit(" . ", 2)
          setup =  joke_division[0]
          punchline = joke_division[1]
      elif news.count('.') == 1 and news.rsplit(".", 1)[1] != ' ':
          joke_division = news.rsplit(" . ", 1)
          setup =  joke_division[0]
          punchline = joke_division[1]
      elif news.count('.') == 1 and news.rsplit(".", 1)[1] == ' ':
          tokenized_text = nltk.word_tokenize(news.lower())
          tokenized_text = [w for w in tokenized_text if w not in stop_symbols]
          setup = " ".join(tokenized_text[:math.ceil(len(tokenized_text) / 2)])
          punchline = " ".join(tokenized_text[math.ceil(len(tokenized_text) / 2):])
  else:
      tokenized_text = nltk.word_tokenize(news.lower())
      tokenized_text = [w for w in tokenized_text if w not in stop_symbols]
      setup = " ".join(tokenized_text[:math.ceil(len(tokenized_text) / 2)])
      punchline = " ".join(tokenized_text[math.ceil(len(tokenized_text) / 2):])
  return setup, punchline

In [ ]:
def clean_all_nouns(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  #tokenized_text = [token.text for token in doc]
  clean_text_l = []
  for token in doc:
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
        continue
      elif token.pos_ != 'NOUN' or token.pos_ != 'PROPN':
        clean_text_l.append(token.text)
  clean_text = " ".join(clean_text_l)
  return clean_text

In [ ]:
ling_df_sec["setup_out"] = [clean_all_nouns(s) for s in ling_df_sec["setup"]]
ling_df_sec["punchline_out"] = [clean_all_nouns(s) for s in ling_df_sec["punchline"]]
ling_df_sec["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_sec["setup_out"], ling_df_sec["punchline_out"])]

In [ ]:
res_df_al_ns = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_all_n =res_df_al_ns.merge(res_df_al_ns.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_all_n = res_df_all_n.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_all_n = res_df_all_n.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_all_n.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_all_noun_1000.csv")

Мера обманутости модели при подаче измененного типа текста


###### Eliminating a single verb.

In [ ]:
ling_df_th = ling_df

In [ ]:
def clean_first_verb(text):
  doc = spacy_model(text)
  tokenized_text = [token.text for token in doc]
  clean_text = []
  for i, token in enumerate(doc):
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.pos_ == 'VERB' or token.pos_ == 'AUX':
        del tokenized_text[i]
        break
      elif token.pos_ != 'VERB' or token.pos_ != 'AUX':
        continue
  clean_text = " ".join(tokenized_text)
  return clean_text

In [ ]:
ling_df_th["setup_out"] = [clean_first_verb(s) for s in ling_df_th["setup"]]
ling_df_th["punchline_out"] = [clean_first_verb(s) for s in ling_df_th["punchline"]]
ling_df_th["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_th["setup_out"], ling_df_th["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_onev =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_onev = res_df_onev.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_onev = res_df_onev.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_onev.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_one_v_1000.csv")

###### Eliminating all verbs.

In [ ]:
ling_df_thh= ling_df

In [ ]:
def clean_all_verbs(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  #tokenized_text = [token.text for token in doc]
  clean_text_l = []
  for token in doc:
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.pos_ == 'VERB' or token.pos_ == 'AUX':
        continue
      elif token.pos_ != 'VERB' or token.pos_ != 'AUX':
        clean_text_l.append(token.text)
  clean_text = " ".join(clean_text_l)
  return clean_text

In [ ]:
ling_df_thh["setup_out"] = [clean_all_verbs(s) for s in ling_df_thh["setup"]]
ling_df_thh["punchline_out"] = [clean_all_verbs(s) for s in ling_df_thh["punchline"]]
ling_df_thh["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_thh["setup_out"], ling_df_thh["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_allv =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_allv = res_df_allv.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_allv = res_df_allv.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

###### Eliminating a single noun phrase with elements linked by syntactic relations nsubj (nominal subject), expl (pleonastic nominations), nmod (attribute or complement in the genitive case), or appos (dependent common noun for modifying the first noun).

In [ ]:
ling_df_for= ling_df

In [ ]:
def clean_first_couple_n_v(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  tokenized_text = [token.text for token in doc]
  clean_text = []

  for i, token in enumerate(doc):
    try:
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.dep_ == 'nsubj' and token.head.pos_=='VERB' and token.head.text in tokenized_text:
        try:
          del tokenized_text[i]
          tokenized_text.remove(token.head.text)
          break
        except IndexError as ie:
          continue
      elif token.dep_ != 'nsubj':
        if token.dep_ == 'nmod' or token.dep_ == 'expl' or token.dep_ == 'appos': # рассмотреть отдельно и со всеми вместе
          try:
            del tokenized_text[i]
            tokenized_text.remove(token.head.text)
          except IndexError as ie:
            continue
        if token.dep_ != 'nmod' or token.dep_ != 'expl' or token.dep_ != 'appos':
          continue
    except ValueError as ver:
      continue
  clean_text = " ".join(tokenized_text)
  return clean_text

In [ ]:
ling_df_for["setup_out"]= [clean_first_couple_n_v(s) for s in ling_df_for["setup"]]
ling_df_for["punchline_out"] = [clean_first_couple_n_v(s) for s in ling_df_for["punchline"] if len(s)]
ling_df_for["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_for["setup_out"], ling_df_for["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_nv =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_nv = res_df_nv.rename(columns={"model_proba": "worst_proba"})


In [ ]:
res_df_nv = res_df_nv.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_nv.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_nsubj.csv")

###### Eliminating a single phrase with elements linked by syntactic relations nsubj (nominal subject).

In [ ]:
ling_df_for= ling_df

In [ ]:
def clean_first_couple_n_v(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  tokenized_text = [token.text for token in doc]
  clean_text = []
  for i, token in enumerate(doc):
    try:
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.dep_ == 'nsubj' and token.head.pos_=='VERB' and token.head.text in tokenized_text:
        del tokenized_text[i]
        tokenized_text.remove(token.head.text)
        break
      elif token.dep_ != 'nsubj':
        continue
    except ValueError as ver:
      continue
  clean_text = " ".join(tokenized_text)
  return clean_text

In [ ]:
ling_df_for["setup_out"]= [clean_first_couple_n_v(s) for s in ling_df_for["setup"]]
ling_df_for["punchline_out"] = [clean_first_couple_n_v(s) for s in ling_df_for["punchline"]]
ling_df_for["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_for["setup_out"], ling_df_for["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_nsubjv =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_nsubjv = res_df_nsubjv.rename(columns={"model_proba": "worst_proba"})


In [ ]:
res_df_nsubjv = res_df_nsubjv.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_nsubjv.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_only_nsubj.csv")

###### Eliminating a single phrase with elements linked by syntactic relations expl (pleonase nominations), nmod (attribute or complement in the genitive case), or appos (dependent common noun for modifying the first noun).

In [ ]:
ling_df_for= ling_df

In [ ]:
def clean_first_couple_n_v(text):
  #for item in tqdm(jokes_df.to_dict(orient='records'), total = len(jokes_df.to_dict(orient='records'))):
  #tokenized_text = word_tokenize(text)
  #tokenized_setup = word_tokenize(item["setup"])
  #tokenized_punchline = word_tokenize(item["punchline"])
  doc = spacy_model(text)
  tokenized_text = [token.text for token in doc]
  clean_text = []
  for i, token in enumerate(doc):
    try:
    #w_tag = nltk.pos_tag([w], tagset='universal')[0][1]
      if token.dep_ == 'nmod' or token.dep_ == 'expl' or token.dep_ == 'appos' and token.head.text in tokenized_text:
        del tokenized_text[i]
        tokenized_text.remove(token.head.text)
        break
      elif token.dep_ != 'nmod' or token.dep_ != 'expl' or token.dep_ != 'appos':
        continue
    except ValueError as ver:
      continue
  clean_text = " ".join(tokenized_text)
  return clean_text

In [ ]:
ling_df_for["setup_out"]= [clean_first_couple_n_v(s) for s in ling_df_for["setup"]]
ling_df_for["punchline_out"] = [clean_first_couple_n_v(s) for s in ling_df_for["punchline"]]
ling_df_for["text_out"] = ["{} {}".format(s,p) for s, p in zip(ling_df_for["setup_out"], ling_df_for["punchline_out"])]

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_setup_punch(setup="setup_out", punchline="punchline")),
                    pd.DataFrame(get_model_proba_setup_punch(setup="setup", punchline="punchline_out")),
                    pd.DataFrame(get_model_proba_text(text="text_out"))])

In [ ]:
res_df_thdep =res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_thdep = res_df_thdep.rename(columns={"model_proba": "worst_proba"})


In [ ]:
res_df_thdep = res_df_thdep.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_thdep.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_three_deps.csv")

###### Eliminating individual joke parts.

In [ ]:
res_df = pd.concat([pd.DataFrame(get_model_proba_text(text="setup")),
                    pd.DataFrame(get_model_proba_text(text="punchline"))])

In [ ]:
res_df_sp_out= res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_sp_out = res_df_sp_out.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_sp_out = res_df_sp_out.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_sp_out.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_setup_punchline_1000.csv")

###### Eliminating individual joke parts.

In [ ]:
res_df = pd.DataFrame(get_model_proba_text(text="setup"))

In [ ]:
res_df_sp_out= res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_sp_out = res_df_sp_out.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_sp_out = res_df_sp_out.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_sp_out.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_setup.csv")

In [ ]:
res_df = pd.DataFrame(get_model_proba_text(text="punchline"))

In [ ]:
res_df_sp_out= res_df.merge(res_df.sort_values(by=["model_proba"], ascending=True).groupby(["id", "text"]).agg({"model_proba":"min", "model_res": "first"}).reset_index().reindex(), on= ["id", "text", "model_proba", "model_res"], how="inner")

res_df_sp_out = res_df_sp_out.rename(columns={"model_proba": "worst_proba"})

In [ ]:
res_df_sp_out = res_df_sp_out.sort_values(by=["label_res", 'id'], ascending=False).reset_index(drop=True).reindex()

In [ ]:
res_df_sp_out.to_csv("/content/gdrive/MyDrive/trained_models/exp_humor_triggers_delete_punchline.csv")